## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-03-51-37 +0000,nyt,"Jerome Cohen, Lawyer Who Saw a Future in China...",https://www.nytimes.com/2025/09/27/world/asia/...
1,2025-09-28-03-50-18 +0000,bbc,"We can pull this round, Starmer says ahead of ...",https://www.bbc.com/news/articles/cn0xzdgyx0do...
2,2025-09-28-03-32-23 +0000,missionlocal,"At Day 200, residents see progress and challen...",https://missionlocal.org/2025/09/sf-16th-stree...
3,2025-09-28-03-29-25 +0000,bbc,Russia has no intention of attacking EU or Nat...,https://www.bbc.com/news/articles/c5ygjv0r2myo...
4,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2291/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
96,trump,41
95,president,8
104,up,7
21,russia,6
144,rally,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
11,2025-09-28-02-26-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,86
176,2025-09-27-06-45-39 +0000,nypost,US to revoke Colombian president’s visa after ...,https://nypost.com/2025/09/27/us-news/us-to-re...,85
43,2025-09-27-22-19-00 +0000,wsj,"Trump Says He Is Ordering Troops to Portland, ...",https://www.wsj.com/us-news/trump-says-he-is-o...,72
57,2025-09-27-21-15-59 +0000,wapo,"Trump calls for troops in Portland, escalating...",https://www.washingtonpost.com/politics/2025/0...,72
34,2025-09-27-23-18-16 +0000,nyt,"Trump Says He Has Ordered Troops to Portland, ...",https://www.nytimes.com/2025/09/27/us/politics...,70


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
11,86,2025-09-28-02-26-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
157,37,2025-09-27-09-30-00 +0000,wsj,A new barrier for foreign workers could open s...,https://www.wsj.com/economy/jobs/trump-h1b-fee...
176,36,2025-09-27-06-45-39 +0000,nypost,US to revoke Colombian president’s visa after ...,https://nypost.com/2025/09/27/us-news/us-to-re...
69,28,2025-09-27-19-50-39 +0000,nypost,US preps for attacks against drug lords deep i...,https://nypost.com/2025/09/27/us-news/us-preps...
15,25,2025-09-28-01-46-27 +0000,nypost,"Stampede at India political rally kills 36, in...",https://nypost.com/2025/09/27/world-news/stamp...
116,24,2025-09-27-13-23-14 +0000,nypost,Cornell law professor demands civil rights pro...,https://nypost.com/2025/09/27/us-news/cornell-...
109,24,2025-09-27-14-47-55 +0000,nypost,Trump orders troop deployment to ‘war ravaged’...,https://nypost.com/2025/09/27/us-news/trump-or...
4,23,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
52,22,2025-09-27-21-38-32 +0000,nypost,"Erika Kirk seen smiling, makes announcement on...",https://nypost.com/2025/09/27/us-news/erika-ki...
130,22,2025-09-27-12-00-00 +0000,latimes,"Lopez: Nine months after the Eaton fire, 'mome...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
